In [1]:
# ────────────────────────────────────────────────
# 0. Imports & configuration
# ────────────────────────────────────────────────
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import TargetEncoder
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 90)
sns.set_style("whitegrid")
%matplotlib inline

In [2]:
# ────────────────────────────────────────────────
# 1. Chargement & premières infos
# ────────────────────────────────────────────────
df = pd.read_csv("../src/data/train.csv") 

In [3]:
# ───────────────────────────────────────────────── 
# 2. Gestion des valeurs manquantes
# ─────────────────────────────────────────────────

# 1 - Imputation de None pour les variables catégorielles manquantes

na_as_none = [
    "Alley","BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2",
    "FireplaceQu","GarageType","GarageFinish","GarageQual","GarageCond",
    "PoolQC","Fence","MiscFeature","MasVnrType", "Electrical"
]
for col in na_as_none:
    if col in df.columns:
        df[col] = df[col].fillna("None")



In [4]:
# 2 - Imputation de la médiane pour les variables numériques manquantes

for col in df.select_dtypes(include=["float64","int64"]).columns:
    df[col] = df[col].fillna(df[col].median())



In [5]:
# 3 - vérification qu'il n'y a plus de valeurs manquantes
print(df.isnull().sum().sum())
print(df.isna().sum().sum())

0
0


In [6]:
# ───────────────────────────────────────────────── 
# 3. gestion de l'encodage des variables catégorielles
# ─────────────────────────────────────────────────

In [7]:
df.select_dtypes(include='object')

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,None,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal
1,RL,Pave,None,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,Gable,CompShg,MetalSd,MetalSd,None,TA,TA,CBlock,Gd,TA,Gd,ALQ,Unf,GasA,Ex,Y,SBrkr,TA,Typ,TA,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal
2,RL,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Mn,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal
3,RL,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,Gable,CompShg,Wd Sdng,Wd Shng,None,TA,TA,BrkTil,TA,Gd,No,ALQ,Unf,GasA,Gd,Y,SBrkr,Gd,Typ,Gd,Detchd,Unf,TA,TA,Y,None,None,None,WD,Abnorml
4,RL,Pave,None,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Av,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,None,TA,TA,PConc,Gd,TA,No,Unf,Unf,GasA,Ex,Y,SBrkr,TA,Typ,TA,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal
1456,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,Gable,CompShg,Plywood,Plywood,Stone,TA,TA,CBlock,Gd,TA,No,ALQ,Rec,GasA,TA,Y,SBrkr,TA,Min1,TA,Attchd,Unf,TA,TA,Y,None,MnPrv,None,WD,Normal
1457,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,Gable,CompShg,CemntBd,CmentBd,None,Ex,Gd,Stone,TA,Gd,No,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Gd,Attchd,RFn,TA,TA,Y,None,GdPrv,Shed,WD,Normal
1458,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,Hip,CompShg,MetalSd,MetalSd,None,TA,TA,CBlock,TA,TA,Mn,GLQ,Rec,GasA,Gd,Y,FuseA,Gd,Typ,None,Attchd,Unf,TA,TA,Y,None,None,None,WD,Normal


In [8]:
df.select_dtypes(include=['int64','float64'])

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2003.0,2,548,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,1976.0,2,460,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2001.0,2,608,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,1998.0,3,642,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,2000.0,3,836,192,84,0,0,0,0,0,12,2008,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,62.0,7917,6,5,1999,2000,0.0,0,0,953,953,953,694,0,1647,0,0,2,1,3,1,7,1,1999.0,2,460,0,40,0,0,0,0,0,8,2007,175000
1456,1457,20,85.0,13175,6,6,1978,1988,119.0,790,163,589,1542,2073,0,0,2073,1,0,2,0,3,1,7,2,1978.0,2,500,349,0,0,0,0,0,0,2,2010,210000
1457,1458,70,66.0,9042,7,9,1941,2006,0.0,275,0,877,1152,1188,1152,0,2340,0,0,2,0,4,1,9,2,1941.0,1,252,0,60,0,0,0,0,2500,5,2010,266500
1458,1459,20,68.0,9717,5,6,1950,1996,0.0,49,1029,0,1078,1078,0,0,1078,1,0,1,0,2,1,5,0,1950.0,1,240,366,0,112,0,0,0,0,4,2010,142125


In [9]:
for col in df.select_dtypes(include='object').columns:
    print(f"{col} : {df[col].nunique()} modalités")

MSZoning : 5 modalités
Street : 2 modalités
Alley : 3 modalités
LotShape : 4 modalités
LandContour : 4 modalités
Utilities : 2 modalités
LotConfig : 5 modalités
LandSlope : 3 modalités
Neighborhood : 25 modalités
Condition1 : 9 modalités
Condition2 : 8 modalités
BldgType : 5 modalités
HouseStyle : 8 modalités
RoofStyle : 6 modalités
RoofMatl : 8 modalités
Exterior1st : 15 modalités
Exterior2nd : 16 modalités
MasVnrType : 4 modalités
ExterQual : 4 modalités
ExterCond : 5 modalités
Foundation : 6 modalités
BsmtQual : 5 modalités
BsmtCond : 5 modalités
BsmtExposure : 5 modalités
BsmtFinType1 : 7 modalités
BsmtFinType2 : 7 modalités
Heating : 6 modalités
HeatingQC : 5 modalités
CentralAir : 2 modalités
Electrical : 6 modalités
KitchenQual : 4 modalités
Functional : 7 modalités
FireplaceQu : 6 modalités
GarageType : 7 modalités
GarageFinish : 4 modalités
GarageQual : 6 modalités
GarageCond : 6 modalités
PavedDrive : 3 modalités
PoolQC : 4 modalités
Fence : 5 modalités
MiscFeature : 5 modali

In [10]:
# 1 - Encodage des variables catégorielles avec OneHotEncoder

cols_to_ohe = [
    "MSSubClass",
    "MSZoning",
    "Street",
    "Alley",
    "LotShape", 
    "LandContour", 
    "Utilities",
    "LotConfig",
    "LandSlope",
    "Condition1",
    "Condition2",
    "BldgType",
    "HouseStyle",
    "RoofStyle",
    "RoofMatl",
    "MasVnrType",
    "BsmtExposure",
    "BsmtFinType1",
    "BsmtFinSF1",
    "BsmtFinType2",
    "Heating",
    "CentralAir",
    "Electrical",
    "Functional",
    "GarageType",
    "PavedDrive",
    "SaleType",
    "SaleCondition",
    "Foundation",
    "Fence",
    "MiscFeature",
]

ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

encoded_to_ohe = ohe.fit_transform(df[cols_to_ohe])

encoded_to_ohe_df = pd.DataFrame(encoded_to_ohe, columns=ohe.get_feature_names_out(cols_to_ohe))

df = pd.concat([df.drop(columns=cols_to_ohe), encoded_to_ohe_df], axis=1)

print(encoded_to_ohe_df)


      MSSubClass_20  MSSubClass_30  MSSubClass_40  MSSubClass_45  \
0               0.0            0.0            0.0            0.0   
1               1.0            0.0            0.0            0.0   
2               0.0            0.0            0.0            0.0   
3               0.0            0.0            0.0            0.0   
4               0.0            0.0            0.0            0.0   
...             ...            ...            ...            ...   
1455            0.0            0.0            0.0            0.0   
1456            1.0            0.0            0.0            0.0   
1457            0.0            0.0            0.0            0.0   
1458            1.0            0.0            0.0            0.0   
1459            1.0            0.0            0.0            0.0   

      MSSubClass_50  MSSubClass_60  MSSubClass_70  MSSubClass_75  \
0               0.0            1.0            0.0            0.0   
1               0.0            0.0            0

In [11]:
df

,Id,LotFrontage,LotArea,Neighborhood,OverallQual,OverallCond,YearBuilt,YearRemodAdd,Exterior1st,Exterior2nd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingQC,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,...,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_None,PavedDrive_N,PavedDrive_P,PavedDrive_Y,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_None,MiscFeature_Gar2,MiscFeature_None,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC
0,1,65.0,8450,CollgCr,7,5,2003,2003,VinylSd,VinylSd,196.0,Gd,TA,Gd,TA,0,150,856,Ex,856,854,0,1710,1,0,2,1,3,1,Gd,8,0,None,2003.0,RFn,2,548,TA,TA,0,61,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,2,80.0,9600,Veenker,6,8,1976,1976,MetalSd,MetalSd,0.0,TA,TA,Gd,TA,0,284,1262,Ex,1262,0,0,1262,0,1,2,0,3,1,TA,6,1,TA,1976.0,RFn,2,460,TA,TA,298,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,3,68.0,11250,CollgCr,7,5,2001,2002,VinylSd,VinylSd,162.0,Gd,TA,Gd,TA,0,434,920,Ex,920,866,0,1786,1,0,2,1,3,1,Gd,6,1,TA,2001.0,RFn,2,608,TA,TA,0,42,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,4,60.0,9550,Crawfor,7,5,1915,1970,Wd Sdng,Wd Shng,0.0,TA,TA,TA,Gd,0,540,756,Gd,961,756,0,1717,1,0,1,0,3,1,Gd,7,1,Gd,1998.0,Unf,3,642,TA,TA,0,35,272,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,5,84.0,14260,NoRidge,8,5,2000,2000,VinylSd,VinylSd,350.0,Gd,TA,Gd,TA,0,490,1145,Ex,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,1,TA,2000.0,RFn,3,836,TA,TA,192,84,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,62.0,7917,Gilbert,6,5,1999,2000,VinylSd,VinylSd,0.0,TA,TA,Gd,TA,0,953,953,Ex,953,694,0,1647,0,0,2,1,3,1,TA,7,1,TA,1999.0,RFn,2,460,TA,TA,0,40,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1456,1457,85.0,13175,NWAmes,6,6,1978,1988,Plywood,Plywood,119.0,TA,TA,Gd,TA,163,589,1542,TA,2073,0,0,2073,1,0,2,0,3,1,TA,7,2,TA,1978.0,Unf,2,500,TA,TA,349,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1457,1458

In [12]:
# 2 - encodage des variables catégorielles avec label

In [13]:
quality_map = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0}
cols_to_label_1 = [
    "ExterQual",
    "ExterCond",
    "BsmtQual",
    "BsmtCond",
    "HeatingQC",
    "KitchenQual",
    "FireplaceQu",
    "GarageQual",
    "GarageCond",
    "PoolQC"
]

for col in cols_to_label_1:
    df[col] = df[col].map(quality_map)



In [14]:
for col in cols_to_label_1:
    print(f"{col} : {df[col].nunique()} modalités")
    print(df[col].value_counts())

ExterQual : 4 modalités
ExterQual
3    906
4    488
5     52
2     14
Name: count, dtype: int64
ExterCond : 5 modalités
ExterCond
3    1282
4     146
2      28
5       3
1       1
Name: count, dtype: int64
BsmtQual : 4 modalités
BsmtQual
3.0    649
4.0    618
5.0    121
2.0     35
Name: count, dtype: int64
BsmtCond : 4 modalités
BsmtCond
3.0    1311
4.0      65
2.0      45
1.0       2
Name: count, dtype: int64
HeatingQC : 5 modalités
HeatingQC
5    741
3    428
4    241
2     49
1      1
Name: count, dtype: int64
KitchenQual : 4 modalités
KitchenQual
3    735
4    586
5    100
2     39
Name: count, dtype: int64
FireplaceQu : 5 modalités
FireplaceQu
4.0    380
3.0    313
2.0     33
5.0     24
1.0     20
Name: count, dtype: int64
GarageQual : 5 modalités
GarageQual
3.0    1311
2.0      48
4.0      14
5.0       3
1.0       3
Name: count, dtype: int64
GarageCond : 5 modalités
GarageCond
3.0    1326
2.0      35
4.0       9
1.0       7
5.0       2
Name: count, dtype: int64
PoolQC : 3 modalit

In [15]:
finished_map = {'Fin': 3,'RFn': 2, 'Unf': 1, 'None': 0}
df["GarageFinish"] = df["GarageFinish"].map(finished_map)


In [16]:
print(df["GarageFinish"].value_counts())

GarageFinish
1    605
2    422
3    352
0     81
Name: count, dtype: int64


In [17]:
# transformations de la variable Sale Price
df["SalePrice"] = np.log1p(df["SalePrice"])

In [18]:
#transformation target encoding avec la variable cible SalePrice

cols_to_target = ["Neighborhood","Exterior1st","Exterior2nd"]
tg_encoder = TargetEncoder(categories='auto', target_type='continuous', smooth='auto', cv=5, random_state=42)
df[cols_to_target] = tg_encoder.fit_transform(df[cols_to_target], df["SalePrice"])
print(df[cols_to_target])


      Neighborhood  Exterior1st  Exterior2nd
0        12.158136    12.212592    12.220251
1        12.288349    11.842247    11.844926
2        12.158136    12.212592    12.220251
3        12.242446    11.810563    11.921561
4        12.692608    12.207681    12.212194
...            ...          ...          ...
1455     12.150562    12.212592    12.220251
1456     12.132217    12.049321    11.998005
1457     12.203268    12.178577    12.181287
1458     11.871430    11.873077    11.877159
1459     11.722215    11.957546    11.982100

[1460 rows x 3 columns]


In [19]:
df.select_dtypes(include='object').columns

Index([], dtype='object')